In [2]:
!pip install numpy==1.26.4
!pip install scikit-learn catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 9.7 MB/s eta 0:00:00m eta 0:00:0100:01:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.2 MB/s eta 0:00:002.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.6 MB/s eta 0:00:007.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 4.7 MB/s eta 0:00:006.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 5.4 MB/s eta 0:00:000:00:010:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import root_mean_squared_error, mean_squared_error


In [5]:
df = pd.read_csv('tmp/data.csv', header=None)

# 重複行を削除
df = df.drop_duplicates()

# 説明変数 (特徴量) と目的変数 (ターゲット) に分ける
# X = df.iloc[:, :-1]  # 最後の列を除いたものが説明変数
# y = df.iloc[:, -1]   # 最後の列が目的変数
X = df.iloc[:, :-1].values  # 最後の列以外が特徴量
y = df.iloc[:, -1].values   # 最後の列が正規化EV

# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# トレーニングデータをCatBoostのPoolに変換
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)


In [11]:
model = CatBoostRegressor(iterations=1500, depth=4, learning_rate=0.1, l2_leaf_reg=7, loss_function='RMSE', eval_metric='RMSE', verbose=500)

# モデルを学習
model.fit(train_pool, eval_set=test_pool)

predictions = model.predict(X_test)



metrics = {
    'RMSE': eval_metric(predictions, y_test, 'RMSE'),
    'MAE': eval_metric(predictions, y_test, 'MAE'),
    'MAPE': eval_metric(predictions, y_test, 'MAPE'),
    'R2': eval_metric(predictions, y_test, 'R2')
}

# 各評価指標を表示
for metric_name, metric_value in metrics.items():
    print(f'{metric_name}: {metric_value}')

# Shrink model to first 1489 iterations.
# RMSE: [0.010081507810215133]
# MAE: [0.007347661887280395]
# MAPE: [0.007348661887280393]
# R2: [0.9823416229149526]


0:	learn: 0.0712025	test: 0.0747232	best: 0.0747232 (0)	total: 1.43ms	remaining: 2.15s
500:	learn: 0.0032227	test: 0.0118061	best: 0.0118061 (500)	total: 248ms	remaining: 494ms
1000:	learn: 0.0016611	test: 0.0111323	best: 0.0111323 (1000)	total: 513ms	remaining: 256ms
1499:	learn: 0.0010805	test: 0.0109508	best: 0.0109502 (1497)	total: 806ms	remaining: 0us

bestTest = 0.01095021089
bestIteration = 1497

Shrink model to first 1498 iterations.
RMSE: [0.010950211340870722]
MAE: [0.006169094505068774]
MAPE: [0.006170093100602041]
R2: [0.9775851561544532]


In [7]:
model.save_model("tmp/bjc.onnx",
           format="onnx",
           export_parameters=None,
           pool=None)